## Final code 

In [ ]:
import tensorflow as tf 
import pandas as pd
import os
import shutil

In [16]:
model = tf.keras.models.load_model("/kaggle/working/bleed.h5")

In [13]:
image_names = []
predicted_labels = []

folder_path = "/kaggle/input/TestDataset"
bleeding_output_dir = "/kaggle/working/Bleeding"
non_bleeding_output_dir = "/kaggle/working/NonBleeding"

# Create a function to load and preprocess an image
def preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0  
    return img

# Get a list of image file names in the folder and sort them
image_file_names = os.listdir(folder_path)
image_file_names.sort()

# Iterate over the images in the folder
for image_name in image_file_names:
    # Remove the file extension (.png) and save only the image name
    image_name_without_extension = os.path.splitext(image_name)[0]
    
    image_path = os.path.join(folder_path, image_name)
    
    # Preprocess the image
    image = preprocess_image(image_path)
    image = tf.expand_dims(image, axis=0)  
    predictions = model.predict(image, verbose=0)
    
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_class = "Bleeding" if predicted_class_index == 0 else "Non-Bleeding"
    
    image_names.append(image_name_without_extension)
    predicted_labels.append(predicted_class)
    if predicted_class == "Bleeding":
        shutil.move(image_path, os.path.join(bleeding_output_dir, image_name))
    else:
        shutil.move(image_path, os.path.join(non_bleeding_output_dir, image_name))

result_df = pd.DataFrame({"Image Name": image_names, "Predicted Label": predicted_labels})

result_df.to_csv("submission.csv", index=False)


In [ ]:
!pip install ultralytics==8.0.134

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=predict model='/kaggle/working/best.pt' conf=0.25 source='/kaggle/working/Bleeding' save=True